In [47]:
import axisrules as axis
import numpy as np
import itertools
rules = [axis.VoterDeletion, axis.BallotCompletion,  axis.ForbiddenTriples]
all_rules = [axis.VoterDeletion, axis.MinimalFlips, axis.BallotCompletion, axis.MinimalSwaps, axis.ForbiddenTriples]

def get_counter_example(m, n):

    for _ in range(100000):
        votes = np.random.randint(0, 2, size=(n, m))
        success = True
        for rule in rules:
            best_axis = rule(votes).bruteforce()
            if len(best_axis) > 1:
                success = False
                break
        if success:
            axis_list = []
            for rule in rules:
                best_axis = rule(votes).bruteforce()
                str_axis = axis.get_axis(best_axis[0][0],  ["a","b","c","d","e","f"][:m])
                if str_axis not in axis_list:
                    axis_list.append(str_axis)
            if len(axis_list) >= 3:
                v = axis.get_profile(votes, ["a","b","c","d","e","f"][:m])
                if len(v) <= 10:
                    print(axis_list)
                    axis.print_profile(votes, ["a","b","c","d","e","f"][:m])
                    return

In [46]:
get_counter_example(4,20)

KeyboardInterrupt: 

In [61]:
path_folder = "C:/Users/teoz0/Documents/Thèse/2024-01/dich-profiles/"
file_name = "dich-profiles-n4-m3"
import os 
# import 
from tabulate import tabulate



def get_counter_example_from_file(n,m):
    candidates = ["a","b","c","d","e","f"][:m]
    file_name = "dich-profiles-n"+str(n)+"-m"+str(m)+".txt"
    path = os.path.join(path_folder, file_name)
    with open(path, "r") as f:
        nseen = 0

        for line in f:
            nseen +=1
            if nseen%20000 == 0:
                print(nseen)
            ballots = line.strip().split(",")
            raw_votes = [[int(y) for y in x.split("+")] for x in ballots]
            seen = False
            for i in range(len(raw_votes)):
                if len(raw_votes[i]) == 1 or len(raw_votes[i]) == m:
                    seen = True
            
            if seen:
                continue 
            votes = np.zeros((n,m))
            for i in range(n):
                for j in raw_votes[i]:
                    votes[i,j] = 1
            res = []
            res_brut = []
            best_vals = []
            for j,rule in enumerate(all_rules):
                best_axis = rule(votes).bruteforce()
                best_vals.append(best_axis[0][1])
                if True:#j not in [1,3]:
                    res.append([axis.get_axis(x[0], candidates) for x in best_axis])
                    res_brut.append([x[0] for x in best_axis])
            
            optis = []
            success = 0
            for i in range(len(res)):
                for k in res[i]:
                    ok = 0
                    for j in range(len(res)):
                        if i != j:
                            if k not in res[j]:
                                ok += 1
                    if ok == len(res)-1:
                        success += 1
                        optis.append(k)
                        break 
                

            if success >= len(res)-1:
                axis_list = []
                for i in range(len(res)):
                    # print(res[i])
                    for x in res[i]:
                        if x not in axis_list:
                            axis_list.append(x)
                # for rule in rules:
                #     best_axis = rule(votes).bruteforce()
                #     str_axis = axis.get_axis(best_axis[0][0], candidates  )
                #     if str_axis not in axis_list:
                #         axis_list.append(str_axis)
                if True:#len(axis_list) >= 5:
                    # print(optis)
                    # axis.print_profile(votes, candidates)
                    # for i in range(len(res)):
                        # print(res[i])
                    all_axes =[]
                    all_axes_str =[]
                    for i in range(len(res)):
                        for x in res_brut[i]:
                            axis_str = axis.get_axis(x, candidates)
                            if axis_str in all_axes_str:
                                continue
                            tab_x = [axis_str]
                            for j,rule in enumerate(all_rules):
                                score = rule(votes).get_score(x)
                                if score == best_vals[j]:
                                    tab_x.append("x")
                                else:
                                    tab_x.append(" ")
                                # tab_x.append(score)
                            all_axes.append(tab_x)
                            all_axes_str.append(axis_str)
                    print(tabulate(all_axes, headers=["axis", "VD", "MF", "BC", "MS", "FT"]))
                


In [62]:
get_counter_example_from_file(6,6)

20000
40000
60000
axis                   VD    MF    BC    MS    FT
---------------------  ----  ----  ----  ----  ----
e < a < b < c < f < d  x     x     x           x
e < a < b < c < d < f  x     x     x           x
b < a < c < e < f < d  x     x     x     x     x
b < a < c < e < d < f  x     x     x     x     x
e < c < a < b < f < d  x     x     x     x     x
e < c < a < b < d < f  x     x     x     x     x
c < b < a < e < f < d  x     x     x           x
c < b < a < e < d < f  x     x     x           x
c < b < a < d < e < f  x
e < f < d < a < b < c  x     x                 x
e < d < f < a < b < c  x     x                 x
d < e < f < a < b < c  x
d < b < a < c < e < f  x
d < e < c < a < b < f  x
e < f < d < c < b < a  x     x
e < d < f < c < b < a  x     x
a < b < c < e < f < d        x
a < b < c < e < d < f        x
e < a < b < f < d < c        x
e < a < b < d < f < c        x
b < a < e < c < f < d        x
b < a < e < c < d < f        x
b < a < e < f < d < c        x
b < a < e <

KeyboardInterrupt: 

In [60]:
def count_lines(n,m):
    candidates = ["a","b","c","d","e","f"][:m]
    file_name = "dich-profiles-n"+str(n)+"-m"+str(m)+".txt"
    path = os.path.join(path_folder, file_name)
    with open(path, "r") as f:
        nseen = 0

        for line in f:
            nseen +=1

    print(nseen)

count_lines(6,6)

223034


In [4]:
import axisrules as axis
import numpy as np
import itertools
all_rules = [axis.VoterDeletion, axis.MinimalFlips, axis.BallotCompletion, axis.MinimalSwaps, axis.ForbiddenTriples]
N= 10000

profile = 18*[[1,1,0,0,0,0,0]] + N*[[0,1,1,0,0,0,0]] + N*[[0,0,1,1,0,0,0]] \
    + 15*[[0,0,0,1,1,0,0]] + 4*[[0,0,0,0,1,1,0]] + 1*[[1,0,0,0,0,0,1]] + 20*[[0,1,1,0,0,1,1]] \
    + 15*[[1,0,0,0,1,1,1]] + 2*[[1,0,0,1,0,0,1]]


In [6]:

for rule in all_rules:
    res = rule(profile).bruteforce()
    print(res[0][1])
    for x in res:
        axis.print_axis(x[0], ["a","b","c","d","e","f","g"])

36
a < e < f < g < b < c < d
37
e < f < g < a < b < c < d
88
e < d < c < b < a < f < g
99
e < d < c < b < f < g < a
128
e < a < f < g < b < c < d


In [8]:
for rule in all_rules:
    print(rule(profile).get_score(x[0]))


40
42
122
122
128
